In [29]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [61]:
options = webdriver.ChromeOptions()
# options.add_argument('headless')

driver = webdriver.Chrome('./chromedriver', options=options)
# driver.get('https://paige.kr.nc.com/community/clubhouse/post/448733') # 홈런볼 잡기 공기
# driver.get('https://paige.kr.nc.com/community/clubhouse/post/357451') # 테스트 아무거나
driver.get('https://paige.kr.nc.com/community/clubhouse/post/394435') # 내 글
action = ActionChains(driver)
time.sleep(1)

# 더 보기 다 눌러놓기
while True:
    time.sleep(1)
    try:
        driver.find_element(By.CLASS_NAME, 'button-more').click()
    except:
        print('더 보기 없음')
        break
        
# 위에서부터 댓글 크롤링
nickname = driver.find_elements(By.CLASS_NAME, 'nickname')
content = driver.find_elements(By.CLASS_NAME, 'comment-content')

#게시글 작성자 이름 빼기
nickname = nickname[1:]

comment_dict = {}

print('닉네임 갯수 : {}'.format(len(nickname)))
print('댓글 갯수 : {}'.format(len(content)))



comment_df = pd.DataFrame(data=[n.text for n in nickname], columns=['닉네임'])
comment_df['댓글'] = [c.text for c in content]
comment_df['Pick_1'] = 0
comment_df['Pick_2'] = 0
comment_df['Pick_3'] = 0

index_count = 0

for c in comment_df['댓글']:
    if len(c.split('/')) == 3:
        comment_df['Pick_1'][index_count] = c.split('/')[0]
        comment_df['Pick_2'][index_count] = c.split('/')[1]
        comment_df['Pick_3'][index_count] = c.split('/')[2]
    index_count +=1

더 보기 없음
닉네임 갯수 : 10
댓글 갯수 : 10


In [62]:
comment_df

,닉네임,댓글,Pick_1,Pick_2,Pick_3
0,주루친스키,김상수/강한울/강민호,김상수,강한울,강민호
1,차명석,김현수/채은성/오지환,김현수,채은성,오지환
2,트레비,나성범/소크라테스/이창진,나성범,소크라테스,이창진
3,키움은선수잘키움,이정후/푸이그/김혜성,이정후,푸이그,김혜성
4,나20대일때롯데5강,김원중/이민석/김도규,김원중,이민석,김도규
5,연탄구이,오지환/김현수/이재원,오지환,김현수,이재원
6,MagicHGS,유강남/오지환/이영빈,유강남,오지환,이영빈
7,minding,정은원/노시환/터크먼,정은원,노시환,터크먼
8,MagicHGS,최강한화 화이팅,0,0,0
9,연탄구이,그러게요,0,0,0


In [58]:
comment_df.to_excel('comment.xlsx')

In [196]:
score_df = pd.read_excel('트리오예측 점수계산.xlsx', sheet_name=[0,1])
accm_df = pd.read_excel('누적 점수 계산시트.xlsx', sheet_name=[0,1], index_col=0)

In [199]:
for i in score_df[1].index:
    try:
        if score_df[1]['선수 이름'][i] in list(accm_df[1]['선수 이름']):
            accm_df[1]['점수'][accm_df[1][accm_df[1]['선수 이름'] == score_df[1]['선수 이름'][i]].index] += score_df[1]['총 점수'][i]
        else:
            df = pd.DataFrame({'선수 이름' : score_df[1]['선수 이름'][i], '점수' : score_df[1]['총 점수'][i]}, index = [0])
            accm_df[1] = accm_df[1].append(df, ignore_index = True)
    except:
        pass

In [202]:
# 결측치 제거
accm_df[1] = accm_df[1].dropna(axis=0)

In [203]:
accm_df[1]

,선수 이름,점수
0,채은성,10.5
1,나성범,15.0
2,김현수,15.0
3,김혜성,15.0
4,소크라테스,18.0
5,오지환,22.5
6,이정후,28.5
7,이창진,10.5
8,푸이그,15.0
9,이재원,15.0


In [204]:
#1. 파일 생성
writer=pd.ExcelWriter('누적 점수 계산시트.xlsx', engine='openpyxl')
 
#2. 생성 파일에 시트명 지정 후 dataframe에 저장한 결과값 넣기
accm_df[0].to_excel(writer, sheet_name='유저 누적')
accm_df[1].to_excel(writer, sheet_name='선수 누적')
 
#3. 작성 완료 후 파일 저장
writer.save()
